# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [3]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [4]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[num_test/2:], newsgroups_test.target[num_test/2:]
dev_data, dev_labels = newsgroups_test.data[:num_test/2], newsgroups_test.target[:num_test/2]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print 'training label shape:', train_labels.shape
print 'test label shape:', test_labels.shape
print 'dev label shape:', dev_labels.shape
print 'labels names:', newsgroups_train.target_names

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

In [8]:
def P1(num_examples=5):
### STUDENT START ###
   print_train_data, print_train_labels = train_data[:num_examples], train_labels[:num_examples]
   for i in range (0,5):
      print '\nData is: ',print_train_data[i], '\nLabel is: ',print_train_labels[i]
   
   

### STUDENT END ###
P1()


Data is:  Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych 
Label is:  1

Data is:  

Seems to be, barring evidence to the contrary, that Koresh was simply
another deranged fanatic who thought it neccessary to take a whole bunch of
folks with him, children and all, to satisfy his delusional mania. Jim
Jones, circa 1993.


Nope - fruitcakes like Koresh have been demonstrating such evil corruption
for centuries. 
Label is:  3

Data is:  

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [14]:
def P2():
### STUDENT START ###
    vectorizer = CountVectorizer(min_df=1)
    sparse_matrix_X = vectorizer.fit_transform(train_data)
    ## Answer a.
    print 'size of vocabulary is ', sparse_matrix_X.size
    print 'Number of non zero features ', sparse_matrix_X.nnz
    print 'Shape of the matrix: ', sparse_matrix_X.shape
    print 'Average number of non-zero features per example is : ', sparse_matrix_X.nnz/sparse_matrix_X.shape[0]
    total_cells = 2034*26879
    print 'Fraction of entries in the matrix that are non-zero is: %10f '  % (sparse_matrix_X.nnz/total_cells)
    
    ## Answer b
    feature_names = vectorizer.get_feature_names()
    print 'The zeroth feature name is: ',feature_names[0], ' and the last feature name is: ',feature_names[len(feature_names)-1]
    
    ## Answer c
    corpus = ["atheism", "graphics", "space", "religion"]
    new_X = vectorizer.fit_transform(corpus)
    ## Confirming the shape of the sparse matrix
    print 'Average number of non-zero features per example is ', new_X.shape[0]/new_X.nnz
    
    ## Answer d
    trigram_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2,3), min_df=1)
    trigram_vectorizer_train = trigram_vectorizer.fit_transform(train_data)
    print 'The vocabulary size is : ',len(trigram_vectorizer.vocabulary_)
    
    
    ## Answer e
    vectorizer_e = CountVectorizer(min_df=10)
    X_e = vectorizer_e.fit_transform(train_data)
    print 'Size of vocab', len(vectorizer_e.vocabulary_)
    
    ## Answer f
    vectorizer_f = CountVectorizer(min_df=1)
    vectorizer_f.fit_transform(train_data)
    train_X_vocab = set(vectorizer_f.vocabulary_.keys())
    vectorizer_f.fit_transform(dev_data)
    dev_X_vocab = set(vectorizer_f.vocabulary_.keys())
    print 'Fraction of words in dev that are missing from train vocab', ( 1.0 * len(dev_X_vocab) - len(train_X_vocab.intersection(dev_X_vocab))) / len(dev_X_vocab)  
### STUDENT END ###
P2()

size of vocabulary is  196700
Number of non zero features  196700
Shape of the matrix:  (2034, 26879)
Average number of non-zero features per example is :  96
Fraction of entries in the matrix that are non-zero is:   0.000000 
The zeroth feature name is:  00  and the last feature name is:  zyxel
Average number of non-zero features per example is  1
The vocabulary size is :  35478
Size of vocab 3064
Fraction of words in dev that are missing from train vocab 0.247876400345


(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [10]:
def P3():
### STUDENT START ###
    pipeline_A = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', KNeighborsClassifier()),
    ])

    pipeline_B = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', MultinomialNB()),
    ])

    pipeline_C = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier',LogisticRegression()),
    ])
    
    parameters_A = {
    'classifier__n_neighbors': (3, 4, 5, 6, 7, 8)
    }
    
    parameters_B = {
    'classifier__alpha': (1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.0)
    }
    
    parameters_C = {
    'classifier__penalty': ('l2','l1'),
    'classifier__C': (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0)
    }
    
    if __name__ == "__main__":
        # multiprocessing requires the fork to happen in a __main__ protected
        # block

        # find the best parameters for both the feature extraction and the
        # classifier
        grid_search_A = GridSearchCV(pipeline_A, parameters_A, n_jobs=-1, verbose=1)

        print("Performing grid search...")
        print("pipeline:", [name for name, _ in pipeline_A.steps])
        print("parameters:")
        print(parameters_A)
        grid_search_A.fit(train_data, train_labels)
        
        print("Best score: %0.3f" % grid_search_A.best_score_)
        print("Best parameters set:")
        best_parameters = grid_search_A.best_estimator_.get_params()
        for param_name in sorted(parameters_A.keys()):
            print("\t%s: %r" % (param_name, best_parameters[param_name]))
            
        
        grid_search_B = GridSearchCV(pipeline_B, parameters_B, n_jobs=-1, verbose=1)

        print("Performing grid search...")
        print("pipeline:", [name for name, _ in pipeline_B.steps])
        print("parameters:")
        print(parameters_B)
        grid_search_B.fit(train_data, train_labels)
        
        print("Best score: %0.3f" % grid_search_B.best_score_)
        print("Best parameters set:")
        best_parameters = grid_search_B.best_estimator_.get_params()
        for param_name in sorted(parameters_B.keys()):
            print("\t%s: %r" % (param_name, best_parameters[param_name]))
            
            
        grid_search_C = GridSearchCV(pipeline_C, parameters_C, n_jobs=-1, verbose=1)

        print("Performing grid search...")
        print("pipeline:", [name for name, _ in pipeline_C.steps])
        print("parameters:")
        print(parameters_C)
        grid_search_C.fit(train_data, train_labels)
        
        
        print("Best score: %0.3f" % grid_search_C.best_score_)
        print("Best parameters set:")
        best_parameters = grid_search_C.best_estimator_.get_params()
        for param_name in sorted(parameters_C.keys()):
            print("\t%s: %r" % (param_name, best_parameters[param_name]))

    
### STUDENT END ###
P3()

Performing grid search...
('pipeline:', ['vectorizer', 'classifier'])
parameters:
{'classifier__n_neighbors': (3, 4, 5, 6, 7, 8)}
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best score: 0.409
Best parameters set:
	classifier__n_neighbors: 5
Performing grid search...
('pipeline:', ['vectorizer', 'classifier'])
parameters:
{'classifier__alpha': (1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.0)}
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    2.0s finished


Best score: 0.811
Best parameters set:
	classifier__alpha: 0.5
Performing grid search...
('pipeline:', ['vectorizer', 'classifier'])
parameters:
{'classifier__C': (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0), 'classifier__penalty': ('l2', 'l1')}
Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    2.0s finished
/home/amitabha/anaconda2/lib/python2.7/site-packages/sklearn/naive_bayes.py:664: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc)
/home/amitabha/anaconda2/lib/python2.7/site-packages/sklearn/naive_bayes.py:664: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc)
/home/amitabha/anaconda2/lib/python2.7/site-packages/sklearn/naive_bayes.py:664: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc)
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   20.8s finished


Best score: 0.776
Best parameters set:
	classifier__C: 0.2
	classifier__penalty: 'l2'


ANSWER:

Optimal Value of K = 5.
Optimal Value of alpha = 0.5.
Optimal Value of C = 0.2 using l2 regularization.

A. Nearest Neighbors doesn't work well here since we have labeled text data and hence the dataset is not continuous in nature.

B. Naive Bayes works better here since there are no corelation between the features.

C. 

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [15]:
def P4():
### STUDENT START ###
    vectorizer = CountVectorizer(ngram_range=(2,2))
    X_data = vectorizer.fit_transform(train_data)
    
    classifier = LogisticRegression(C=0.5)
    classifier.fit(X_data,train_labels)
    
    vocab_index_arr = []
    vocabularies = vectorizer.get_feature_names()
    
    # Couldn't print the table properly
    
### STUDENT END ###
P4()

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [52]:
def empty_preprocessor(s):
    return s

def better_preprocessor(s):
### STUDENT START ###
    s = re.sub('[^(a-zA-ZNUM)]', ' ',s)
### STUDENT END ###

def P5():
### STUDENT START ###
    norm_vectorizer_5 = CountVectorizer()
    preprocess_vectorizer_5 = CountVectorizer(preprocessor=better_preprocessor(_))
    norm_train_5 = norm_vectorizer_5.fit_transform(train_data)
    preprocess_train_5 = preprocess_vectorizer_5.fit_transform(train_data)
    
    
    norm_dev_5 = norm_vectorizer_5.transform(dev_data)
    norm_logreg = LogisticRegression(C=0.5)
    norm_logreg.fit(norm_train_5,train_labels)
    
    preprocess_dev_5 = preprocess_vectorizer_5.transform(dev_data)
    preprocess_logreg = LogisticRegression(C=0.5)
    preprocess_logreg.fit(preprocess_train_5,train_labels)
    
    norm_predicted = norm_logreg.predict(norm_dev_5)
    preprocess_predicted = preprocess_logreg.predict(preprocess_dev_5)
    print 'F1 score without using preprocessor is : ',metrics.f1_score(dev_labels,norm_predicted)
    print 'F1 score without analyzer of char is : ',metrics.f1_score(dev_labels,preprocess_predicted)
    
### STUDENT END ###
P5()

F1 score without using preprocessor is :  0.708473977649
F1 score without analyzer of char is :  0.708473977649


/home/amitabha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
/home/amitabha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

In [15]:
def P6():
    # Keep this random seed here to make comparison easier.
    np.random.seed(0)

    ### STUDENT START ###
    
    ### STUDENT END ###
P6()

(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [66]:
def P7():
### STUDENT START ###
    vectorizer_7 = TfidfVectorizer()
    vect_train = vectorizer_7.fit_transform(train_data)
    # Using C=100
    logreg = LogisticRegression(C=100)
    # Fitting Training data
    logreg.fit(vect_train,train_labels)
    # Transforming Dev data
    vect_dev = vectorizer_7.transform(dev_data)
    # Predicting on Dev data
    prediction = logreg.predict(vect_dev)
    
    print "For Tfidf with C=100 F1 score is : ",metrics.f1_score(dev_labels,prediction)
    
    predict_prob = logreg.predict_proba(vect_dev)
    for i in range(len(dev_labels)):
        ratios = (max(predict_prob[i])/predict_prob[i][dev_labels[i]])
        print ratios
## STUDENT END ###
P7()

For Tfidf with C=100 F1 score is :  0.759766242785
1.0
1.0
1.0
1.0
1.0
1.0
1.03446512487
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
8.0811107913
1.49477628476
1.0
1.0
1.0
1.0
5.18482416157
4.32863348286
1.21808460106
1.0
1.0
1.0
1.0
1.0
1.49477628476
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
7.29501510691
1.0
1.0
1.0
4.54001973861
4.91421581688
1.0
1.0
1.0
1.0
49.561220189
1.0
1.0
6.23080141406
1.0
4.10367333156
1.0
1.0
6.63911695773
2.80143255235
1.65307371251
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.00994584699
1.0
1.0
1.0
1.0
1.0
1.0
1.0
13.2435502269
1.0
1.0
1.0
1.54357760476
1.0
1.0
8.90589126428
34.5712907758
1.0
1.0
25.2152212334
1.0
1.0
1.0
1.0
1.1298911053
1.0
1.0
2.81114174159
1.0
1.0
1.0
1.0
1.19707594746
1.0
1.0
1.0
23.3814398724
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.98695700597
1.34184509556
1.0
35.8708439692
1.0
1.0
1.0
1.0
3.14079317186
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
33.2506850317
1.0
1.0
1.0
1.0
1.0
1.0
29.308576734
18.0856951351
7.98915688834
1.0
1.0
1

/home/amitabha/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.